# 1 nowy plik na bazie starego pliku (wszystkie dane domyslne) - najnowsza wersja 2025.08.12

In [1]:
import os
import re
import time
import win32com.client
from win32com.client import constants as xlc

# ----------------- KONFIGURACJA -----------------
source_folder = r"C:\Users\a\Desktop\directory_with_changed_files"
prefix = "copied_"
# Jeśli chcesz najpierw testować na jednym pliku, podaj jego pełną ścieżkę tutaj:
TEST_FILE = None  # np. r"C:\Users\a\Desktop\directory_with_changed_files\example.xlsx"
# ------------------------------------------------

def make_unique_path(path):
    if not os.path.exists(path):
        return path
    base, ext = os.path.splitext(path)
    i = 1
    new_path = f"{base}_{i}{ext}"
    while os.path.exists(new_path):
        i += 1
        new_path = f"{base}_{i}{ext}"
    return new_path

def get_fileformat_by_ext(ext):
    ext = ext.lower()
    if ext == ".xlsm":
        return 52
    if ext == ".xlsx":
        return 51
    if ext == ".xls":
        return 56
    return 51

def reset_codenames(wb, excel_app):
    """
    Dodaje tymczasowy moduł VBA i zmienia CodeName arkuszy na Sheet1, Sheet2, ...
    Zwraca True jeśli operacja wykonana poprawnie, False w przeciwnym razie.
    """
    try:
        vb_module = wb.VBProject.VBComponents.Add(1)
        try:
            vb_module.Name = f"tmp_reset_codenames_{int(time.time())}"
        except Exception:
            pass

        macro_code = (
            "Sub ResetCodenames()\n"
            "    Dim i As Long\n"
            "    On Error Resume Next\n"
            "    For i = 1 To ActiveWorkbook.Sheets.Count\n"
            "        ActiveWorkbook.VBProject.VBComponents(ActiveWorkbook.Sheets(i).CodeName).Name = \"Sheet\" & i\n"
            "    Next i\n"
            "    On Error GoTo 0\n"
            "End Sub\n"
        )

        vb_module.CodeModule.AddFromString(macro_code)
        macro_fullname = f"'{wb.Name}'!ResetCodenames"
        excel_app.Run(macro_fullname)
        wb.VBProject.VBComponents.Remove(vb_module)
        return True
    except Exception as e:
        print(f"⚠️ Nie udało się ustawić CodeName arkuszy (VBProject niedostępny lub inny błąd): {e}")
        return False

def break_external_links_fast(wb_target, wb_source_path, excel_app):
    """
    Szybsza, bezpieczniejsza wersja naprawiania linków:
    - Replace tylko w komórkach zawierających formuły (SpecialCells(xlCellTypeFormulas))
    - Replace na całych arkuszach (Range.Replace) aby uniknąć pętli po komórkach
    - Aktualizacja Names, BreakLink dla Excel links
    """
    report = {"formulas_replaced": 0, "names_fixed": 0, "links_broken": 0, "errors": []}
    orig_name = os.path.basename(wb_source_path)

    try:
        # przygotuj możliwe warianty do zastąpienia
        literal_fullpath = wb_source_path
        bracket_token = f"[{orig_name}]"

        for sht in wb_target.Sheets:
            try:
                # używamy SpecialCells żeby trafić tylko komórki z formułami
                try:
                    formulas = sht.UsedRange.SpecialCells(xlc.xlCellTypeFormulas)
                except Exception:
                    formulas = None

                if formulas is not None:
                    # Replace pełnej ścieżki jeśli występuje
                    try:
                        if literal_fullpath:
                            formulas.Replace(literal_fullpath, "")
                            formulas.Replace(f"'{literal_fullpath}'", "")
                    except Exception:
                        pass
                    # Replace tokenu [filename]
                    try:
                        formulas.Replace(bracket_token, "")
                    except Exception:
                        pass
                    # usuń podwójne apostrofy
                    try:
                        formulas.Replace("''", "'")
                    except Exception:
                        pass
                    # nie liczymy dokładnie ile komórek zmieniono (COM nie zwraca takiej liczby),
                    # inkrementujemy raz na arkusz jako sygnał wykonania
                    report["formulas_replaced"] += 1

            except Exception as e:
                report["errors"].append(f"Sheet {getattr(sht, 'Name', '?')} replace error: {e}")

        # Names
        try:
            for nm in list(wb_target.Names):
                try:
                    refers = nm.RefersTo
                    if refers and (orig_name in refers or (literal_fullpath and literal_fullpath in refers)):
                        new_ref = refers.replace(literal_fullpath, "").replace(bracket_token, "").replace("''", "'")
                        nm.RefersTo = new_ref
                        report["names_fixed"] += 1
                except Exception:
                    pass
        except Exception as e:
            report["errors"].append(f"Names processing error: {e}")

        # LinkSources i BreakLink (xlLinkTypeExcelLinks = 1)
        try:
            links = None
            try:
                links = wb_target.LinkSources(1)
            except Exception:
                links = None
            if links:
                # iteracja po linkach - COM może zwrócić listę lub pojedynczy string
                try:
                    for link in links:
                        try:
                            wb_target.BreakLink(Name=link, Type=1)
                            report["links_broken"] += 1
                        except Exception as ex:
                            report["errors"].append(f"Cannot break link: {link} -> {ex}")
                except TypeError:
                    # jeśli links to pojedynczy string
                    try:
                        wb_target.BreakLink(Name=links, Type=1)
                        report["links_broken"] += 1
                    except Exception as ex:
                        report["errors"].append(f"Cannot break link: {links} -> {ex}")
        except Exception as e:
            report["errors"].append(f"LinkSources/BreakLink error: {e}")

    except Exception as e:
        report["errors"].append(str(e))

    return report

def copy_excel_with_formats(file_path, excel_app):
    file_name = os.path.basename(file_path)
    print(f"\n--- START: {file_name} ---")
    if file_name.startswith(prefix):
        print(f"ℹ️ Pomijam już skopiowany plik: {file_name}")
        return

    output_dir = os.path.dirname(file_path)
    output_file = os.path.join(output_dir, prefix + file_name)
    output_file = make_unique_path(output_file)

    # zapisz bieżące ustawienia aplikacji by je przywrócić
    prev_screen = getattr(excel_app, "ScreenUpdating", True)
    try:
        prev_calc = excel_app.Calculation
    except Exception:
        prev_calc = None

    try:
        # ustawienia przyspieszające
        try:
            excel_app.ScreenUpdating = False
        except Exception:
            pass
        try:
            excel_app.Calculation = xlc.xlCalculationManual
        except Exception:
            pass
        excel_app.DisplayAlerts = False

        # Otwórz źródło w trybie tylko do odczytu (nie aktualizujemy linków podczas otwarcia)
        wb_source = excel_app.Workbooks.Open(file_path, ReadOnly=True, UpdateLinks=0)
        # Kopiujemy wszystkie arkusze do nowego workbooka
        wb_source.Sheets.Copy()
        wb_target = excel_app.ActiveWorkbook

        # reset codenames (jeśli możliwe)
        codenames_ok = reset_codenames(wb_target, excel_app)
        if codenames_ok:
            base_out, ext_out = os.path.splitext(output_file)
            if ext_out.lower() != ".xlsm":
                output_file = make_unique_path(base_out + ".xlsm")
                ff = get_fileformat_by_ext(".xlsm")
            else:
                ff = get_fileformat_by_ext(ext_out)
        else:
            _, ext = os.path.splitext(output_file)
            ff = get_fileformat_by_ext(ext)

        # Naprawa zewnętrznych odwołań (szybsza wersja)
        report = break_external_links_fast(wb_target, file_path, excel_app)
        if report["formulas_replaced"] or report["names_fixed"] or report["links_broken"]:
            print(f"ℹ️ Naprawiono odwołania (arkusze z formułami przetworzone): {report}")
        if report["errors"]:
            print("⚠️ Niektóre elementy mogły nie zostać naprawione:", report["errors"])

        # SaveAs i zamknięcie
        wb_target.SaveAs(output_file, FileFormat=ff)
        wb_target.Close(SaveChanges=False)
        wb_source.Close(SaveChanges=False)
        print(f"✅ Utworzono kopię: {output_file} (CodeName zmienione: {codenames_ok})")

    except Exception as e:
        print(f"❌ Błąd przy pliku {file_name}: {e}")
        # sprzątanie
        try:
            for wb in excel_app.Workbooks:
                try:
                    wb.Close(SaveChanges=False)
                except Exception:
                    pass
        except Exception:
            pass
    finally:
        # przywróć ustawienia excela
        try:
            if prev_calc is not None:
                excel_app.Calculation = prev_calc
        except Exception:
            pass
        try:
            excel_app.ScreenUpdating = prev_screen
        except Exception:
            pass
        excel_app.DisplayAlerts = False
        print(f"--- KONIEC: {file_name} ---\n")

def process_all_xlsx(root_folder, test_file=None):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False
    excel.DisplayAlerts = False
    try:
        if test_file:
            if os.path.exists(test_file):
                copy_excel_with_formats(test_file, excel)
            else:
                print(f"Test file nie istnieje: {test_file}")
            return

        for dirpath, dirnames, filenames in os.walk(root_folder):
            for fname in filenames:
                if fname.startswith("~$"):
                    continue
                if fname.startswith(prefix):
                    continue
                lower = fname.lower()
                if not (lower.endswith(".xlsx") or lower.endswith(".xlsm") or lower.endswith(".xls")):
                    continue
                full_path = os.path.join(dirpath, fname)
                copy_excel_with_formats(full_path, excel)
    finally:
        try:
            excel.Quit()
        except Exception:
            pass

if __name__ == "__main__":
    # Jeśli chcesz przetestować 1 plik ustaw TEST_FILE powyżej
    process_all_xlsx(source_folder, test_file=TEST_FILE)



--- START: Future Tech_ASTM -Indir A_C.xlsx ---
⚠️ Nie udało się ustawić CodeName arkuszy (VBProject niedostępny lub inny błąd): (-2147352567, 'Wystąpił wyjątek.', (0, 'Microsoft Excel', 'Programmatic access to Visual Basic Project is not trusted\n', 'xlmain11.chm', 0, -2146827284), None)
ℹ️ Naprawiono odwołania (arkusze z formułami przetworzone): {'formulas_replaced': 17, 'names_fixed': 0, 'links_broken': 0, 'errors': []}
✅ Utworzono kopię: C:\Users\a\Desktop\directory_with_changed_files\Klienci\2025\Aalberts\copied_Future Tech_ASTM -Indir A_C.xlsx (CodeName zmienione: False)
--- KONIEC: Future Tech_ASTM -Indir A_C.xlsx ---


--- START: Innovatest_ASTM -Indir A_C_15Y.xlsx ---
⚠️ Nie udało się ustawić CodeName arkuszy (VBProject niedostępny lub inny błąd): (-2147352567, 'Wystąpił wyjątek.', (0, 'Microsoft Excel', 'Programmatic access to Visual Basic Project is not trusted\n', 'xlmain11.chm', 0, -2146827284), None)
ℹ️ Naprawiono odwołania (arkusze z formułami przetworzone): {'formulas_

# 2 Zmiana daty utworzenia, modyfikacji, etc

In [2]:
import os
import zipfile
import tempfile
import random
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import pywintypes
import win32file
import win32con
import win32com.client

def losowa_data_utworzenia():
    start = datetime(2024, 12, 15)
    end = datetime(2025, 5, 31)
    delta = end - start
    return start + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def losowa_data_modyfikacji_wzgledem_utworzenia(data_utw):
    min_data = data_utw + timedelta(days=5)
    max_data = data_utw + timedelta(days=20)
    delta = max_data - min_data
    return min_data + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def losowa_data_drukowania():
    start = datetime(2025, 8, 1)
    end = datetime(2025, 12, 10)
    delta = end - start
    return start + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def format_data_iso(dt):
    return dt.isoformat(timespec='seconds') + "Z"

def ustaw_systemowa_date_pliku(file_path, utw_dt, mod_dt):
    wintime_utw = pywintypes.Time(utw_dt)
    wintime_mod = pywintypes.Time(mod_dt)
    handle = win32file.CreateFile(
        file_path, win32con.GENERIC_WRITE, 0, None,
        win32con.OPEN_EXISTING, win32con.FILE_ATTRIBUTE_NORMAL, None)
    win32file.SetFileTime(handle, wintime_utw, None, wintime_mod)
    handle.close()

def modyfikuj_corexml(path, utw_dt, mod_dt, print_dt):
    ns = {
        'cp': 'http://schemas.openxmlformats.org/package/2006/metadata/core-properties',
        'dc': 'http://purl.org/dc/elements/1.1/',
        'dcterms': 'http://purl.org/dc/terms/',
        'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
    }
    tree = ET.parse(path)
    root = tree.getroot()

    def wyczysc(tag):
        el = root.find(tag, ns)
        if el is not None:
            el.text = ""

    wyczysc('dc:title')
    wyczysc('dc:subject')
    wyczysc('cp:manager')
    wyczysc('cp:company')

    if (el := root.find('dc:creator', ns)) is not None:
        el.text = "Anonim"
    if (el := root.find('cp:lastModifiedBy', ns)) is not None:
        el.text = "Anonim"
    if (el := root.find('dcterms:created', ns)) is not None:
        el.text = format_data_iso(utw_dt)
    if (el := root.find('dcterms:modified', ns)) is not None:
        el.text = format_data_iso(mod_dt)

    last_printed = root.find('cp:lastPrinted', ns)
    if last_printed is None:
        last_printed = ET.SubElement(root, '{http://schemas.openxmlformats.org/package/2006/metadata/core-properties}lastPrinted')
    last_printed.text = format_data_iso(print_dt)

    tree.write(path, encoding='utf-8', xml_declaration=True)

def modyfikuj_appxml(path):
    try:
        tree = ET.parse(path)
        root = tree.getroot()
        for tag in ['Company', 'Manager', 'Application', 'Title', 'Subject']:
            el = root.find(tag)
            if el is not None:
                el.text = ""
        tree.write(path, encoding='utf-8', xml_declaration=True)
    except:
        pass

def przetworz_xlsx(file_path, output_path, utw_dt, mod_dt, print_dt):
    with tempfile.TemporaryDirectory() as tempdir:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(tempdir)

        core_path = os.path.join(tempdir, 'docProps', 'core.xml')
        app_path = os.path.join(tempdir, 'docProps', 'app.xml')

        if os.path.exists(core_path):
            modyfikuj_corexml(core_path, utw_dt, mod_dt, print_dt)
        if os.path.exists(app_path):
            modyfikuj_appxml(app_path)

        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as new_zip:
            for foldername, subfolders, filenames in os.walk(tempdir):
                for filename in filenames:
                    full_path = os.path.join(foldername, filename)
                    rel_path = os.path.relpath(full_path, tempdir)
                    new_zip.write(full_path, rel_path)

    ustaw_systemowa_date_pliku(output_path, utw_dt, mod_dt)

def konwertuj_xls_na_xlsm(excel, file_path):
    output_path = file_path + "m"
    wb = excel.Workbooks.Open(file_path, False, False, None, None, None, None, None, None, True)  # Open as editable
    wb.SaveAs(output_path, FileFormat=52)  # 52 = .xlsm (z makrami)
    wb.Close()
    return output_path

def przetworz_folder_rekursywnie(folder_path):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.DisplayAlerts = False
    excel.Visible = False

    for root, dirs, files in os.walk(folder_path):
        for fname in files:
            full_path = os.path.join(root, fname)

            # Konwersja .xls → .xlsm (zachowując makra)
            if fname.endswith(".xls") and not fname.endswith(".xlsx"):
                try:
                    print(f"🔄 Konwertuję .xls → .xlsm: {fname}")
                    full_path = konwertuj_xls_na_xlsm(excel, full_path)
                    fname = os.path.basename(full_path)
                except Exception as e:
                    print(f"⚠️ Błąd konwersji .xls → .xlsm: {fname} → {e}")
                    continue

            if fname.endswith(".xlsx") or fname.endswith(".xlsm"):
                if "_random_date" in fname:
                    continue

                utw_dt = losowa_data_utworzenia()
                mod_dt = losowa_data_modyfikacji_wzgledem_utworzenia(utw_dt)
                print_dt = losowa_data_drukowania()

                new_name = fname.replace(".xlsx", "_random_date.xlsx").replace(".xlsm", "_random_date.xlsm")
                output_path = os.path.join(root, new_name)

                try:
                    przetworz_xlsx(full_path, output_path, utw_dt, mod_dt, print_dt)
                    print(f"✅ {fname} → {os.path.basename(output_path)}")
                    print(f"   Utworzono: {utw_dt.date()} | Zmodyfikowano: {mod_dt.date()} | Drukowano: {print_dt.date()}")
                except Exception as e:
                    print(f"❌ Błąd przetwarzania {fname}: {e}")

    excel.Quit()


# 🔽 Ścieżka główna (można podać dowolną lokalizację)
folder = r"C:\Users\a\Desktop\directory_with_changed_files"
przetworz_folder_rekursywnie(folder)


✅ copied_Future Tech_ASTM -Indir A_C.xlsx → copied_Future Tech_ASTM -Indir A_C_random_date.xlsx
   Utworzono: 2025-02-08 | Zmodyfikowano: 2025-02-15 | Drukowano: 2025-11-12
✅ copied_Innovatest_ASTM -Indir A_C_15Y.xlsx → copied_Innovatest_ASTM -Indir A_C_15Y_random_date.xlsx
   Utworzono: 2025-02-05 | Zmodyfikowano: 2025-02-21 | Drukowano: 2025-10-17
✅ copied_Presi_ASTM - Indir HV 1_10.xlsx → copied_Presi_ASTM - Indir HV 1_10_random_date.xlsx
   Utworzono: 2025-05-23 | Zmodyfikowano: 2025-06-07 | Drukowano: 2025-08-16
✅ Future Tech_ASTM -Indir A_C.xlsx → Future Tech_ASTM -Indir A_C_random_date.xlsx
   Utworzono: 2025-05-29 | Zmodyfikowano: 2025-06-17 | Drukowano: 2025-12-08
✅ Innovatest_ASTM -Indir A_C_15Y.xlsx → Innovatest_ASTM -Indir A_C_15Y_random_date.xlsx
   Utworzono: 2025-01-13 | Zmodyfikowano: 2025-01-31 | Drukowano: 2025-11-09
✅ Presi_ASTM - Indir HV 1_10.xlsx → Presi_ASTM - Indir HV 1_10_random_date.xlsx
   Utworzono: 2025-04-19 | Zmodyfikowano: 2025-04-27 | Drukowano: 2025-11